In [1]:
import os
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import smoother
from smoother import SpatialWeightMatrix, SpatialLoss
from smoother.models.deconv import LinearRegression, NNLS, NuSVR, DWLS, LogNormReg

In [2]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [3]:
data_dir = "/Users/jfr2137/Desktop/Research/Rabadan Lab/Spatial Deconvolution/Data/ductal_carcinoma/"
res_dir = "/Users/jfr2137/Desktop/Research/Rabadan Lab/Spatial Deconvolution/Results/ductal_carcinoma/"

### Load 10x Visium data:

In [4]:
def read_sp_data(data_dir, sample_id):
    from matplotlib.image import imread
    import json
    count_dir = data_dir + "filtered_count_matrix/"
    sp_dir = data_dir + "spatial/"
    meta_dir = data_dir + "metadata.csv"
    
    # load counts
    adata = sc.read_10x_mtx(count_dir)
    
    # load spatial data
    adata.uns["spatial"] = dict()
    adata.uns["spatial"][sample_id] = dict()

    files = dict(
        tissue_positions_file = sp_dir + '/tissue_positions_list.csv',
        scalefactors_json_file = sp_dir + '/scalefactors_json.json',
        hires_image = sp_dir + '/tissue_hires_image.png',
        lowres_image = sp_dir  + '/tissue_lowres_image.png',
    )

    adata.uns["spatial"][sample_id]['images'] = dict()

    for res in ['hires', 'lowres']:
        try:
            adata.uns["spatial"][sample_id]['images'][res] = imread(
                str(files[f'{res}_image'])
            )
        except Exception:
            raise OSError(f"Could not find '{res}_image'")

    # read json scalefactors
    with open(files['scalefactors_json_file']) as f:
        js = json.load(f)
    adata.uns["spatial"][sample_id]['scalefactors'] = js

    # read coordinates
    positions = pd.read_csv(files['tissue_positions_file'], header=None)
    positions.columns = [
        'barcode',
        'in_tissue',
        'array_row',
        'array_col',
        'pxl_col_in_fullres',
        'pxl_row_in_fullres',
    ]
    positions.index = positions['barcode']

    adata.obs = adata.obs.join(positions, how="left")

    adata.obsm['spatial'] = adata.obs[
        ['pxl_row_in_fullres', 'pxl_col_in_fullres']
    ].to_numpy()
    adata.obs.drop(
        columns=['barcode', 'pxl_row_in_fullres', 'pxl_col_in_fullres'],
        inplace=True,
    )
    
    # load meta
    meta = pd.read_csv(meta_dir, header=0, sep=',', index_col=0)
    adata.obs = adata.obs.join(meta, how="left")
    
    return adata

In [5]:
sample_id = "ductal_carcinoma"
adata = read_sp_data(data_dir+"10x_visium/", sample_id)

### Load and reformat data:

#### Subset top 20 markers for each cluster (from Seurat sc_breast_cancer_preprocessing.Rmd script):

In [6]:
df_marker_genes = pd.read_csv(os.path.join(data_dir, "scref", "cluster_all_markers.csv"))
df_marker_genes = df_marker_genes.loc[df_marker_genes["avg_log2FC"] > 0,:]
# df_marker_genes = df_marker_genes.drop_duplicates(subset=["gene"], keep=False)

In [7]:
df_marker_genes = df_marker_genes.groupby(['cluster'])
df_marker_genes = df_marker_genes.apply(lambda x: x.sort_values(["avg_log2FC"], ascending=False))
df_marker_genes = df_marker_genes.reset_index(drop=True)
df_marker_genes = df_marker_genes.groupby('cluster').head(20)

In [8]:
df_marker_genes.to_csv(os.path.join(data_dir, "scref", "top20_markergenes.csv"), index=False)

In [9]:
# Sanity check that the markers look correct:
df_marker_genes[df_marker_genes["cluster"] == "B-cells"]

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
0,1.477579e-76,6.471349,0.328,0.007,2.681953e-72,B-cells,VPREB3
1,3.492989e-60,6.193916,0.231,0.005,6.340124e-56,B-cells,FCRLA
2,2.128491e-74,6.154821,0.284,0.007,3.863425e-70,B-cells,CD19
3,1.874988e-110,6.030925,0.408,0.011,3.403291e-106,B-cells,BANK1
4,1.053964e-191,5.949531,0.633,0.016,1.913050e-187,B-cells,MS4A1
5,1.478044e-53,5.819526,0.232,0.008,2.682798e-49,B-cells,TNFRSF13C
6,8.556241e-71,5.609067,0.277,0.017,1.553043e-66,B-cells,LINC00926
7,1.533925e-50,5.433835,0.201,0.006,2.784228e-46,B-cells,BLK
8,5.216369e-210,4.962387,0.742,0.048,9.468230e-206,B-cells,CD79A
9,1.144403e-39,4.781557,0.201,0.013,2.077206e-35,B-cells,FCRL5


In [10]:
df_marker_genes = df_marker_genes.loc[:, ["gene"]]
df_marker_genes = df_marker_genes.reset_index(drop=True)
df_marker_genes

,gene
0,VPREB3
1,FCRLA
2,CD19
3,BANK1
4,MS4A1
...,...
175,TRAT1
176,CAMK4
177,IL2RB
178,IFNG


#### Normalize spatial data (follow the same procedure as Seurat default):

In [11]:
df_st_expression = adata.to_df().T
df_st_expression = df_st_expression.reset_index(drop=False).rename(columns={"index": "gene"})

In [12]:
gene_col = df_st_expression.iloc[:,0]
sum_counts = df_st_expression.iloc[:,1:].sum(axis=0)
normalized_counts = df_st_expression.iloc[:,1:].div(sum_counts, axis = 1)
normalized_counts = normalized_counts * 10000
normalized_counts = np.log1p(normalized_counts)
df_st_expression = pd.concat([gene_col, normalized_counts], axis = 1)

In [13]:
df_st_expression = df_marker_genes.merge(df_st_expression, how="left")
df_st_expression = df_st_expression.drop(columns=["gene"])
df_st_expression

,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATGGTGAGAGGA-1,AAACATTTCCCGGATT-1,AAACCCGAACGAAATC-1,...,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGGCAATGACTG-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.341417,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.911073,0.0,0.000000,0.000000,0.0,0.0
178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0


In [14]:
df_cluster_expression = pd.read_csv(os.path.join(data_dir, "scref", "cluster_average_exp.csv"))
df_cluster_expression = df_marker_genes.merge(df_cluster_expression, how="left")
df_cluster_expression = df_cluster_expression.drop(columns=["gene"])
df_cluster_expression

,Endothelial,CAFs,PVL,B.cells,T.cells,Myeloid,Normal.Epithelial,Plasmablasts,Cancer.Epithelial
0,0.064721,0.020843,0.293500,2.476632,0.003324,0.008216,0.001178,0.039788,0.005644
1,0.044260,0.018353,0.130966,1.235733,0.002830,0.010935,0.000911,0.062973,0.001429
2,0.047473,0.013023,0.136632,1.604874,0.006258,0.015840,0.001779,0.153662,0.002859
3,0.102813,0.031013,0.420284,2.870765,0.009819,0.039606,0.007300,0.026686,0.005384
4,0.357502,0.095500,1.456816,10.117938,0.100093,0.052850,0.009217,0.071286,0.011121
...,...,...,...,...,...,...,...,...,...
175,0.021833,0.034143,0.105066,0.159187,1.335091,0.018592,0.005414,0.313750,0.011514
176,0.017960,0.048506,0.094623,0.172367,1.127906,0.013021,0.007320,0.180962,0.013173
177,0.013216,0.038816,0.073222,0.140745,1.095516,0.010462,0.011580,0.262797,0.013790
178,0.050470,0.151870,0.360431,1.168689,4.462997,0.021167,0.007901,1.004215,0.011229


### Run Smoother:

In [15]:
y = torch.tensor(df_st_expression.values).float()
x = torch.tensor(df_cluster_expression.values).float()

In [16]:
df_metadata = pd.read_csv(data_dir+"10x_visium/ifdata.csv")
df_metadata = df_metadata.rename(columns = {"spot": "voxel"})

In [17]:
pixels = adata.obsm['spatial']
w_raw = SpatialWeightMatrix()
w_raw.calc_weights_knn(pixels, k=6)

Number of spots: 4727. Average number of neighbors per spot:  6.00.


In [18]:
torch.manual_seed(20220722)

weights_dict = {
    "raw": w_raw,
#     "expr_c": w_expr_c,
#     "expre_e": w_expr_e,
#     "hist_m": w_hist_m,
#     "hist_pca": w_hist_pca,
#     "hc_e": w_hc_e,
#     "hc_h": w_hc_h
}

methods_dict = {
    "NNLS": NNLS,
    "NuSVR": NuSVR,
    "DWLS": DWLS,
    "LR": LinearRegression
}

#### Run Smoother using spatial (raw) and no spatial:

In [19]:
list_weights = ["raw", "no_spatial"]

for weight_str in list_weights:
    if weight_str == "no_spatial":
        spatial_loss = None
    else:
        spatial_loss = SpatialLoss('icar', weights_dict[weight_str])
    for method_str in methods_dict.keys():
        model_spatial = methods_dict[method_str]()
        model_spatial.deconv(x, y, spatial_loss=spatial_loss, max_epochs=2000, 
            lambda_spatial_loss=3, lr=1e-3, init_with_lr_sol=True, verbose=False)
        df_spatial = pd.DataFrame(model_spatial.get_props(), columns=df_cluster_expression.columns)
        df_spatial["voxel"] = adata.obs["in_tissue"].index.to_list()
        df_spatial = df_spatial.merge(df_metadata)
        df_spatial.to_csv(f"{res_dir}deconv_results/{sample_id}_{weight_str}_{method_str}.csv", index=False)

=== Time  26.51s. Total epoch 411. Final loss: (total) 0.313. (spatial) 0.001.
=== Time  68.75s. Total epoch 1098. Final loss: (total) 0.125. (spatial) 0.000.
=== Time  14.66s. Total epoch 284. Final loss: (total) 1.271. (spatial) 0.002.
=== Time  27.11s. Total epoch 526. Final loss: (total) 0.304. (spatial) 0.001.
=== Time  14.89s. Total epoch 389. Final loss: (total) 0.312. (spatial) 0.000.
=== Time  60.20s. Total epoch 1105. Final loss: (total) 0.124. (spatial) 0.000.
=== Time  17.65s. Total epoch 335. Final loss: (total) 1.261. (spatial) 0.000.
=== Time  33.83s. Total epoch 521. Final loss: (total) 0.302. (spatial) 0.000.


### Reformatting for ggplot:

In [21]:
df_spatial_nnls = pd.read_csv(f"{res_dir}deconv_results/ductal_carcinoma_raw_NNLS.csv")
df_spatial_nnls = df_spatial_nnls.loc[(~pd.isna(df_spatial_nnls["count"])) & (df_spatial_nnls["annotation"].isin(("Invasive", "Benign hyperplasia"))), :]
df_spatial_nnls.loc[:, "x_adjusted"] = 1 - (df_spatial_nnls["x"] / 24240)
df_spatial_nnls.loc[:, "y_adjusted"] = df_spatial_nnls["y"] / 24240
df_spatial_nnls.to_csv(f"{res_dir}nnls_tcell_cd3_results.csv", index=False)

In [22]:
df_spatial_card = pd.read_csv(f"{res_dir}card_results/ductal_carcinoma_card_spatial.csv")
df_spatial_card = df_spatial_card.rename(columns={"Unnamed: 0": "voxel"})
df_spatial_card = df_spatial_card.merge(df_metadata, how="inner")
df_spatial_card = df_spatial_card.loc[(~pd.isna(df_spatial_card["count"])) & (df_spatial_card["annotation"].isin(("Invasive", "Benign hyperplasia"))), :]
df_spatial_card.loc[:, "x_adjusted"] = 1 - (df_spatial_card["x"] / 24240)
df_spatial_card.loc[:, "y_adjusted"] = df_spatial_card["y"] / 24240
df_spatial_card.to_csv(f"{res_dir}card_tcell_cd3_results.csv", index=False)

### Reformatting CARD results for Python plotting:

In [23]:
df_spatial_card = pd.read_csv(f"{res_dir}card_results/ductal_carcinoma_card_spatial.csv")
df_spatial_card = df_spatial_card.rename(columns={"Unnamed: 0": "voxel"})
df_spatial_card = df_spatial_card.merge(df_metadata, how="inner")
df_spatial_card.columns = [col.replace("-", ".").replace(" ", ".") for col in df_spatial_card.columns]
df_spatial_card.to_csv(f"{res_dir}deconv_results/ductal_carcinoma_raw_CARD.csv", index=False)
df_spatial_card

,voxel,Endothelial,CAFs,PVL,B.cells,T.cells,Myeloid,Normal.Epithelial,Plasmablasts,Cancer.Epithelial,x,y,cd3_intensity,dapi_intensity,count,annotation
0,AAACAAGTATCTCCCA-1,0.037902,0.015373,0.000549,4.006851e-07,4.553169e-06,0.136718,0.035339,0.001597,0.772518,15140,8285,0.062099,0.067375,23.0,Invasive
1,AAACAATCTACTAGCA-1,0.000124,0.003080,0.000325,7.806969e-08,1.565152e-06,0.080196,0.006895,0.009123,0.900256,4029,16386,0.100454,0.196626,NaN,Non-tumor
2,AAACACCAATAACTGC-1,0.008871,0.019838,0.001066,2.763763e-07,3.942342e-05,0.003475,0.094576,0.001715,0.870419,17332,19591,0.043342,0.152094,23.0,Invasive
3,AAACAGAGCGACTCCT-1,0.001029,0.002112,0.002082,1.733133e-09,2.300840e-07,0.000405,0.060811,0.000521,0.933040,6604,9419,0.080191,0.033493,NaN,In situ
4,AAACAGCTTTCAGAAG-1,0.015047,0.145898,0.005655,4.193612e-06,1.529694e-04,0.327339,0.024785,0.004255,0.476864,13543,20974,0.058305,0.064497,18.0,Invasive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4720,TTGTTTCACATCCAGG-1,0.000788,0.028320,0.000891,7.306540e-08,1.481234e-05,0.001251,0.048217,0.000177,0.920341,17079,16456,0.047842,0.086842,19.0,Invasive
4721,TTGTTTCATTAGTCTA-1,0.011388,0.105883,0.015518,6.909669e-08,2.754988e-05,0.009414,0.107004,0.000154,0.750611,17562,18090,0.058697,0.101880,18.0,Invasive
4722,TTGTTTCCATACAACT-1,0.001437,0.005891,0.000138,4.987140e-07,3.193811e-05,0.017229,0.059844,0.001536,0.913893,14005,18517,0.044723,0.085940,24.0,Invasive
4723,TTGTTTGTATTACACG-1,0.004212,0.132797,0.062168,2.217542e-07,6.631463e-05,0.031862,0.164769,0.041573,0.562553,20639,16575,0.041553,0.066416,26.0,Invasive


In [24]:
df_nospatial_card = pd.read_csv(f"{res_dir}card_results/ductal_carcinoma_card_nospatial.csv")
df_nospatial_card = df_nospatial_card.rename(columns={"Unnamed: 0": "voxel"})
df_nospatial_card = df_nospatial_card.merge(df_metadata, how="inner")
df_nospatial_card.columns = [col.replace("-", ".").replace(" ", ".") for col in df_nospatial_card.columns]
df_nospatial_card.to_csv(f"{res_dir}/deconv_results/ductal_carcinoma_no_spatial_CARD.csv", index=False)
df_nospatial_card

,voxel,Endothelial,CAFs,PVL,B.cells,T.cells,Myeloid,Normal.Epithelial,Plasmablasts,Cancer.Epithelial,x,y,cd3_intensity,dapi_intensity,count,annotation
0,AAACAAGTATCTCCCA-1,0.037457,0.015492,0.000544,3.747201e-07,4.205327e-06,0.136850,0.035398,0.001584,0.772670,15140,8285,0.062099,0.067375,23.0,Invasive
1,AAACAATCTACTAGCA-1,0.000124,0.003084,0.000323,7.254467e-08,1.412840e-06,0.080230,0.007008,0.008931,0.900300,4029,16386,0.100454,0.196626,NaN,Non-tumor
2,AAACACCAATAACTGC-1,0.008894,0.019805,0.001048,2.600018e-07,3.724121e-05,0.003547,0.092291,0.001732,0.872646,17332,19591,0.043342,0.152094,23.0,Invasive
3,AAACAGAGCGACTCCT-1,0.000988,0.002052,0.001989,1.523797e-09,1.963975e-07,0.000397,0.061051,0.000483,0.933039,6604,9419,0.080191,0.033493,NaN,In situ
4,AAACAGCTTTCAGAAG-1,0.015089,0.145953,0.005623,3.985119e-06,1.479557e-04,0.327495,0.024290,0.004222,0.477176,13543,20974,0.058305,0.064497,18.0,Invasive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4720,TTGTTTCACATCCAGG-1,0.000786,0.028152,0.000879,6.799522e-08,1.399263e-05,0.001256,0.046780,0.000176,0.921956,17079,16456,0.047842,0.086842,19.0,Invasive
4721,TTGTTTCATTAGTCTA-1,0.011458,0.106200,0.015461,6.356209e-08,2.636963e-05,0.009605,0.104455,0.000154,0.752640,17562,18090,0.058697,0.101880,18.0,Invasive
4722,TTGTTTCCATACAACT-1,0.001433,0.005858,0.000135,4.669562e-07,2.952923e-05,0.017213,0.058522,0.001530,0.915278,14005,18517,0.044723,0.085940,24.0,Invasive
4723,TTGTTTGTATTACACG-1,0.004217,0.132864,0.062183,2.076524e-07,6.329674e-05,0.032300,0.162269,0.042103,0.564000,20639,16575,0.041553,0.066416,26.0,Invasive
